# ML-Enhanced European Option Pricing: Data Fetching and Processing

Complete data pipeline from fetching raw options market data and preparing it for machine learning modeling.

## Pipeline Overview:
1. **Data Fetching**: Options data from Yahoo Finance (real-time market data)
2. **Data Cleaning**: Remove invalid entries and apply quality filters
3. **Feature Engineering**: Create additional features for modeling
4. **Black-Scholes Calculation**: Compute theoretical prices
5. **Target Variable**: Calculate pricing errors (Market Price - BS Price)
6. **Data Quality Assessment**: Check for outliers and data quality issues
7. **Normalization**: Standardize features for modeling
8. **Data Export**: Save processed data for modeling


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sys.path.append('..')

from config import Config
from src.data_fetcher import OptionsDataFetcher
from src.black_scholes import BlackScholesCalculator
from src.data_processor import OptionsDataProcessor

plt.style.use('default')
sns.set_palette("husl")

print("Setup complete")


Setup complete


## Step 1: Configuration and Setup

In [2]:
Config.validate_config()
print(f"Target: {Config.TARGET_SYMBOL}")

fetcher = OptionsDataFetcher()
processor = OptionsDataProcessor()
bs_calculator = BlackScholesCalculator()

print("Components ready")

INFO:src.data_fetcher:OptionsDataFetcher initialized successfully


Target: SPY
Components ready


## Step 2: Data Fetching

In [3]:
# Fetch market data
print("📈 Fetching market data...")

risk_free_rate = fetcher.get_risk_free_rate(Config.RISK_FREE_RATE_TICKER)
stock_price = fetcher.get_current_stock_price(Config.TARGET_SYMBOL)

print(f"✅ Risk-free rate: {risk_free_rate:.4f} ({risk_free_rate*100:.2f}%)")
print(f"✅ Current {Config.TARGET_SYMBOL} price: ${stock_price:.2f}")

# Fetch historical-style options data from Yahoo Finance
print(f"\n📊 Fetching historical-style options data for {Config.TARGET_SYMBOL}...")
options_list = fetcher.fetch_historical_style_options(Config.TARGET_SYMBOL)

if not options_list:
    print("❌ Failed to fetch options data")
    raise Exception("Data fetching failed")

print(f"✅ Successfully fetched {len(options_list)} options contracts")


📈 Fetching market data...


INFO:src.data_fetcher:Risk-free rate fetched: 0.0409
INFO:src.data_fetcher:Current SPY price: $645.31
INFO:src.data_fetcher:Fetching historical-style options data for SPY...
INFO:src.data_fetcher:Fetching options data for SPY from Yahoo Finance...
INFO:src.data_fetcher:Found 8 short-term expiration dates
INFO:src.data_fetcher:Using 8 expiration dates, targeting 62 contracts per date
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-08-25


✅ Risk-free rate: 0.0409 (4.09%)
✅ Current SPY price: $645.31

📊 Fetching historical-style options data for SPY...


INFO:src.data_fetcher:Fetching options chain for expiration: 2025-08-26
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-08-27
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-08-28
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-08-29
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-09-05
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-09-12
INFO:src.data_fetcher:Fetching options chain for expiration: 2025-09-19
INFO:src.data_fetcher:Successfully fetched 496 options contracts representing historical-style data


✅ Successfully fetched 496 options contracts


In [4]:
# Process options data
print("🔄 Processing options data...")
raw_df = fetcher.process_options_data(options_list, stock_price, risk_free_rate)

print(f"\n📊 Raw Data Summary:")
print(f"Shape: {raw_df.shape}")
print(f"Columns: {list(raw_df.columns)}")
print(f"\nFirst few rows:")
print(raw_df.head())

# Save raw data
raw_data_path = f"{Config.DATA_RAW_PATH}options_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
os.makedirs(Config.DATA_RAW_PATH, exist_ok=True)
fetcher.save_raw_data(raw_df, raw_data_path)
print(f"\n💾 Raw data saved to: {raw_data_path}")


INFO:src.data_fetcher:Processed 496 option contracts into DataFrame
INFO:src.data_fetcher:Raw data saved to data/raw/options_data_20250823_163952.csv


🔄 Processing options data...

📊 Raw Data Summary:
Shape: (496, 18)
Columns: ['symbol', 'underlying_symbol', 'strike', 'expiration_date', 'contract_type', 'implied_volatility', 'open_interest', 'volume', 'bid', 'ask', 'last_price', 'market_price', 'underlying_price', 'risk_free_rate', 'fetch_timestamp', 'time_to_expiry', 'data_source', 'simulated_trade_date']

First few rows:
                 symbol underlying_symbol  strike expiration_date  \
0  SPY20250825C00530000               SPY   530.0      2025-08-25   
1  SPY20250825C00550000               SPY   550.0      2025-08-25   
2  SPY20250825C00570000               SPY   570.0      2025-08-25   
3  SPY20250825C00580000               SPY   580.0      2025-08-25   
4  SPY20250825C00590000               SPY   590.0      2025-08-25   

  contract_type  implied_volatility  open_interest  volume     bid     ask  \
0          call            0.841798           11.0    11.0  113.98  116.78   
1          call            0.703128            2.0 

## Step 3: Data Cleaning and Quality Assessment

In [5]:
# Create configuration for data cleaning
cleaning_config = {
    'MIN_VOLUME': Config.MIN_VOLUME,
    'MIN_OPEN_INTEREST': Config.MIN_OPEN_INTEREST,
    'MIN_TIME_TO_EXPIRY': Config.MIN_TIME_TO_EXPIRY
}

print("🧹 Starting data cleaning process...")
print(f"Cleaning configuration: {cleaning_config}")

# Clean the data
cleaned_df = processor.clean_raw_data(raw_df, cleaning_config)

# Display cleaning results
print(f"\n📈 Data Cleaning Results:")
print(f"Original shape: {raw_df.shape}")
print(f"Cleaned shape:  {cleaned_df.shape}")
print(f"Rows removed:   {len(raw_df) - len(cleaned_df)} ({(len(raw_df) - len(cleaned_df))/len(raw_df)*100:.1f}%)")

# Check for missing values
print(f"\n🔍 Missing Values Check:")
missing_summary = cleaned_df.isnull().sum()
if missing_summary.sum() > 0:
    print(missing_summary[missing_summary > 0])
else:
    print("✅ No missing values found!")
    
print(f"\n📊 Cleaned Data Sample:")
print(cleaned_df.head())


INFO:src.data_processor:Data cleaning complete. Final shape: (457, 19)
INFO:src.data_processor:Removed 39 rows during cleaning


🧹 Starting data cleaning process...
Cleaning configuration: {'MIN_VOLUME': 0, 'MIN_OPEN_INTEREST': 1, 'MIN_TIME_TO_EXPIRY': 0.002}

📈 Data Cleaning Results:
Original shape: (496, 18)
Cleaned shape:  (457, 19)
Rows removed:   39 (7.9%)

🔍 Missing Values Check:
✅ No missing values found!

📊 Cleaned Data Sample:
                 symbol underlying_symbol  strike expiration_date  \
0  SPY20250825C00530000               SPY   530.0      2025-08-25   
1  SPY20250825C00550000               SPY   550.0      2025-08-25   
2  SPY20250825C00570000               SPY   570.0      2025-08-25   
3  SPY20250825C00580000               SPY   580.0      2025-08-25   
4  SPY20250825C00590000               SPY   590.0      2025-08-25   

  contract_type  implied_volatility  open_interest  volume     bid     ask  \
0          call            0.841798           11.0    11.0  113.98  116.78   
1          call            0.703128            2.0     2.0   93.98   96.79   
2          call            0.565434     

In [6]:
# Save cleaned data
if len(cleaned_df) > 0:
    cleaned_data_path = f"{Config.DATA_PROCESSED_PATH}cleaned_options_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    os.makedirs(Config.DATA_PROCESSED_PATH, exist_ok=True)
    processor.save_processed_data(cleaned_df, cleaned_data_path)
    print(f"\n💾 Cleaned data saved to: {cleaned_data_path}")
else:
    print(f"\n⚠️ No data to save - all records were filtered out during cleaning")


💾 Cleaned data saved to: data/processed/cleaned_options_data_20250823_164021.csv
